<a href="https://colab.research.google.com/github/pollinations/hive/blob/main/interesting_notebooks/Interactive_JS_Button_at_Runtime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Method to inject data into a running python loop using a listen server subprocess
by [@remi_durant](https://twitter/remi_durant)


In [ ]:
#@title HTML/JS Based Remote Control - This needs to be run first!
#@markdown Once the server is running, any other cells that were run during this session can send data to it, allowing for remote control! This cell has no python in it, but it still needs to be run fresh so that Colab can proxy the request to localhost properly.

%%html
<button type="button" onclick="tell('hello')">Ping Server</button>
<button type="button" onclick="tell('stop')">Stop Server</button>
<script>
function tell(data)
{
    var xhr = new XMLHttpRequest();
    xhr.open("PUT", "http://localhost:42", true);
    xhr.setRequestHeader("Content-Type", "application/json");
    xhr.onreadystatechange = function () {
    if (xhr.readyState === 4) {
        console.log(xhr.status);
        console.log(xhr.responseText);
    }};
    if ( typeof(data) !== 'object' || Array.isArray(data) )
        data = { 'msg': data }                
    xhr.send(JSON.stringify(data));
}
</script>

In [ ]:
#@title Processing Loop and Listen Server (for client data injection)
#@markdown This launches a listen server in a subprocess which can then be accessed by the client javascript through http requests! This allows for input from the client while a cell is actively running, for example allowing to change behavior in the middle of an optimization loop!

from IPython import display 

from multiprocessing import Process, Pipe

import socket
from six.moves import socketserver, SimpleHTTPServer

import json

class V6Server(socketserver.TCPServer):
    address_family = socket.AF_INET6
    allow_reuse_address = True 
    logging = False

class Handler(SimpleHTTPServer.SimpleHTTPRequestHandler):

    def do_PUT(self):        
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()
        
        data = self.rfile.read(int(self.headers['Content-Length']))
        data = json.loads(data)
        
        self.server.pipe.send(data)

        if data.get('msg') == 'stop':
            raise KeyboardInterrupt

    def log_message(self, format, *args):
        if self.server.logging:
            SimpleHTTPServer.SimpleHTTPRequestHandler.log_message(self, format, *args)

def js_listener(pipe):
    httpd = V6Server(('::', port), Handler)
    httpd.pipe = pipe
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        pass
    httpd.server_close()

if __name__ == '__main__':
  try:
      
      port = 42 # portpicker.pick_unused_port()
      js = '''
              function tell(data)
              {
                  var xhr = new XMLHttpRequest();
                  xhr.open("PUT", "http://localhost:''' + str(port) + '''", true);
                  xhr.setRequestHeader("Content-Type", "application/json");
                  xhr.onreadystatechange = function () {
                  if (xhr.readyState === 4) {
                      console.log(xhr.status);
                      console.log(xhr.responseText);
                  }};
                  if ( typeof(data) !== 'object' || Array.isArray(data) )
                      data = { 'msg': data }                
                  xhr.send(JSON.stringify(data));
              }
          '''
          
      display.display( display.Javascript(js))    
      display.display( display.HTML( '<button type="button" onclick="tell(\'hi\')">Test Server</button>' ))

      pipe, child_pipe = Pipe()
      p = Process(target=js_listener, args=(child_pipe,))
      p.start()
      
      print('Waiting for Input')
      while True:
          if pipe.poll():
              r = pipe.recv()
              print(f'Received Message: {r}')
              if r.get('msg') == 'stop':
                break

  except KeyboardInterrupt:
    pass

<IPython.core.display.Javascript object>

Waiting for Input
Received Message: {'msg': 'hi'}
Received Message: {'msg': 'hello'}
Received Message: {'msg': 'hello'}
Received Message: {'msg': 'stop'}
